# Capstone Project - Short Answer Grading

In [1]:
# source code for SKLearn custom train.py
!pygmentize source/train.py

from __future__ import print_function

import argparse
import os
import pandas as pd
from source.utils import generate_data

from sklearn.externals import joblib
from keras.layers.core import Dropout
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier


# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")

    # load using joblib
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    print("Done loading model.")

    return model


if __name__ == '__main__':
    # All of the model parameters and training parameters are sent as arguments
    # when this script is executed, during a training job

    # Here we set up an argument parser to easily access

# Setup AWS

In [3]:
import pandas as pd
import boto3
import sagemaker

# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

In [6]:
# should be the name of directory you created to save your features data
data_dir = 'data/seb'

# set prefix, a descriptive name for a directory  
prefix = 'sagemaker/short_answer'

# upload all data to S3
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

# Training Basic Model

In [10]:
from sagemaker.sklearn.estimator import SKLearn
# instantiate a pytorch estimator
# estimator = SKLearn(entry_point='train.py',
#                     source_dir='source', # this should be just "source" for your code
#                     role=role,
#                     train_instance_count=1,
#                     train_instance_type='ml.c4.xlarge',
#                     hyperparameters = {
#                         'epochs': 200,
#                         'embedding_size': 30,
#                         'lstm_size': 100,
#                         'dropout': 0.2,
#                         'optimizer': 'adam'
#                     }
#                    )

In [17]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='source/train.py', 
                       role=role,
                       train_instance_count=1, 
                       train_instance_type='ml.c4.xlarge',
                       framework_version='1.12.0', 
                       py_version='py3',
                       script_mode=True,
                       hyperparameters = {
                        'epochs': 200,
                        'embedding_size': 30,
                        'lstm_size': 100,
                        'dropout': 0.2,
                        'optimizer': 'adam'
                    })

In [34]:
from tensorflow.python.training.adam import AdamOptimizer

In [113]:
%%time
# Train estimator on S3 training data
estimator.fit(input_data)

2019-06-25 21:06:22 Starting - Starting the training job...
2019-06-25 21:06:25 Starting - Launching requested ML instances......
2019-06-25 21:07:32 Starting - Preparing the instances for training......
2019-06-25 21:08:46 Downloading - Downloading input data
2019-06-25 21:08:46 Training - Downloading the training image..
2019-06-25 21:09:03,625 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-06-25 21:09:03,631 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-06-25 21:09:03,956 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-06-25 21:09:03,972 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-06-25 21:09:03,984 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "current_host": "algo-1",
    "fra

## Low Level Training Job

In [ ]:
# We will need to know the name of the container that we want to use for training. SageMaker provides
# a nice utility method to construct this for us.
container = get_image_uri(session.boto_region_name, 'xgboost')

# We now specify the parameters we wish to use for our training job
training_params = {}

# We need to specify the permissions that this training job will have. For our purposes we can use
# the same permissions that our current SageMaker session has.
training_params['RoleArn'] = role

# Here we describe the algorithm we wish to use. The most important part is the container which
# contains the training code.
training_params['AlgorithmSpecification'] = {
    "TrainingImage": container,
    "TrainingInputMode": "File"
}

# We also need to say where we would like the resulting model artifacts stored.
training_params['OutputDataConfig'] = {
    "S3OutputPath": "s3://" + session.default_bucket() + "/" + prefix + "/output"
}

# We also need to set some parameters for the training job itself. Namely we need to describe what sort of
# compute instance we wish to use along with a stopping condition to handle the case that there is
# some sort of error and the training script doesn't terminate.
training_params['ResourceConfig'] = {
    "InstanceCount": 1,
    "InstanceType": "ml.m4.xlarge",
    "VolumeSizeInGB": 5
}
    
training_params['StoppingCondition'] = {
    "MaxRuntimeInSeconds": 86400
}

# Next we set the algorithm specific hyperparameters. In this case, since we are setting up
# a training job which will serve as the base training job for the eventual hyperparameter
# tuning job, we only specify the _static_ hyperparameters. That is, the hyperparameters that
# we do _not_ want SageMaker to change.
training_params['StaticHyperParameters'] = {
    "gamma": "4",
    "subsample": "0.8",
    "objective": "reg:linear",
    "early_stopping_rounds": "10",
    "num_round": "200"
}

# Now we need to tell SageMaker where the data should be retrieved from.
training_params['InputDataConfig'] = [
    {
        "ChannelName": "train",
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": train_location,
                "S3DataDistributionType": "FullyReplicated"
            }
        },
        "ContentType": "csv",
        "CompressionType": "None"
    },
    {
        "ChannelName": "validation",
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": val_location,
                "S3DataDistributionType": "FullyReplicated"
            }
        },
        "ContentType": "csv",
        "CompressionType": "None"
    }
]


=[
    {
        "Name": "loss",
        "Regex": "Loss = (.*?);",
    },
    {
        "Name": "ganloss",
        "Regex": "GAN_loss=(.*?);",
    },    
    {
        "Name": "discloss",
        "Regex": "disc_train_loss=(.*?);",
    },    
    {
        "Name": "disc-combined",
        "Regex": "disc-combined=(.*?);",
    },    
]

## Hypertuning 

In [111]:
# First, make sure to import the relevant objects used to construct the tuner
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

tf_hyperparameter_tuner = HyperparameterTuner(estimator = estimator, # The estimator object to use as the basis for the training jobs.
                                               objective_metric_name = 'Validation_loss', # The metric used to compare trained models.
                                               objective_type = 'Minimize', # Whether we wish to minimize or maximize the metric.
                                               max_jobs = 6, # The total number of models to train
                                               max_parallel_jobs = 3, # The number of models to train in parallel
                                               hyperparameter_ranges = {
                                                    'epochs': IntegerParameter(50, 300),
                                                    'dropout': ContinuousParameter(0.05, 0.5),
                                                    'embeding_size': IntegerParameter(10, 200),
                                                    'lstm_size': IntegerParameter(10, 200)
                                               })


In [106]:
print('{}/train.csv'.format(input_data))

s3://sagemaker-us-east-1-399712746635/sagemaker/short_answer/train.csv


In [109]:
s3_input_train = sagemaker.s3_input(s3_data='{}/train.csv'.format(input_data), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='{}/test.csv'.format(input_data), content_type='csv')

In [112]:
tf_hyperparameter_tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

ClientError: An error occurred (ValidationException) when calling the CreateHyperParameterTuningJob operation: A metric is required for this hyperparameter tuning job objective. Provide a metric in the metric definitions.

In [ ]:
tf_hyperparameter_tuner.wait()

In [67]:
predictor = estimator.deploy(initial_instance_count=1,
                             # instance_type='ml.c5.xlarge',
                             instance_type='ml.t2.medium',
                             endpoint_type='tensorflow-serving')

---------------------------------------------------------------------------------------------------!

In [52]:
import os

job_name = estimator._current_job_name
print(job_name)
print(bucket)
model_data = os.path.join('s3://',bucket,job_name,'output/model.tar.gz')
print(model_data)

sagemaker-tensorflow-scriptmode-2019-06-24-21-36-44-186
sagemaker-us-east-1-399712746635
s3://sagemaker-us-east-1-399712746635/sagemaker-tensorflow-scriptmode-2019-06-24-21-36-44-186/output/model.tar.gz


In [ ]:
from sagemaker.tensorflow.serving import Model
model = Model(model_data=model_data, role=role)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')

In [ ]:
from sagemaker.tensorflow import TensorFlowModel

tf_model = TensorFlowModel(model_data=model_data,
                           role=role,
                           py_version='py3',
                           entry_point='source/train.py',
                           name='short-answer')

predictor = tf_model.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge')

In [90]:
import numpy as np
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # rounding and squeezing array
    test_preds = np.squeeze(predictor.predict(test_features)['predictions'])
    # Normalized to range 0 to 1 if needed
    if test_preds.max() > 1:
        test_preds = test_preds * (1 / test_preds.max())
    test_preds = np.round(test_preds)    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # print metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actuals'], colnames=['predictions']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return tp, fp, fn, tn, precision, recall, accuracy

In [70]:
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)


In [97]:
test_y = test_data.iloc[:, 0].values
test_x = test_data.iloc[:, 1:].values

In [98]:
# First: generate predicted, class labels
test_y_preds = np.squeeze(predictor.predict(test_x)['predictions'])
#normalize if needed
if max(test_y_preds) > 1:
    test_y_preds = test_y_preds * (1 / test_y_preds.max())
print(test_y_preds)    

[-0.01876325  0.02068142  0.02653523  0.35195724  0.72061773  0.87722571
  0.01853903  0.26202348 -0.14137819  0.08305078  0.0955543   0.01703948
  0.04691585  0.13622449  0.99101351  0.87536935  1.          0.80075494
  0.02298404  0.029142    0.0258112   0.29992913  0.23560735  0.03663976
  0.55473981  0.45330276  0.02148174  0.36156186  0.15412955  0.07393319
  0.03006211  0.02122484  0.92390228  0.02723624  0.02116076  0.04249387
  0.03247419  0.02789197  0.0264619   0.06763694  0.20393909  0.03341093
  0.8354164 ]


In [100]:
# Second: calculate the test accuracy
tp, fp, fn, tn, precision, recall, accuracy = evaluate(predictor, test_x, test_y)

print(accuracy)

## print out the array of predicted and true labels, if you want
results = pd.concat([pd.DataFrame(test_y_preds), pd.DataFrame(np.round(test_y_preds)), pd.DataFrame(test_y)], axis=1)
results.columns = ['raw_predicted','predicted','actual']
# print('\nPredicted class labels: ')
# print(test_y_preds)
# print('\nTrue class labels: ')
# print(test_y.values)
print(results)

predictions  -0.0   1.0
actuals                
0.0            23     3
1.0            11     6

Recall:     0.353
Precision:  0.667
Accuracy:   0.674

0.6744186046511628
    raw_predicted  predicted  actual
0       -0.018763       -0.0     0.0
1        0.020681        0.0     1.0
2        0.026535        0.0     0.0
3        0.351957        0.0     0.0
4        0.720618        1.0     1.0
5        0.877226        1.0     1.0
6        0.018539        0.0     0.0
7        0.262023        0.0     0.0
8       -0.141378       -0.0     1.0
9        0.083051        0.0     0.0
10       0.095554        0.0     1.0
11       0.017039        0.0     0.0
12       0.046916        0.0     0.0
13       0.136224        0.0     0.0
14       0.991014        1.0     1.0
15       0.875369        1.0     0.0
16       1.000000        1.0     1.0
17       0.800755        1.0     0.0
18       0.022984        0.0     0.0
19       0.029142        0.0     1.0
20       0.025811        0.0     0.0
21       0.2999